# Medlock Matrix
Using Medlock's[16] method for conversion from Mossong[own directory]

In [37]:
countries = ['BE', 'DE', 'FI', 'GB', 'IT', 'LU', 'NL', 'PL']

ages_dist = pd.read_excel('./data/mossong/age_distributions.xlsx', sheetname='census')
# ages_dist

In [50]:
# Create the normalized symmetric Medlock Matrix
M = np.zeros((15,15))
for country in countries:
    C = pd.read_excel('./data/mossong/original_mossong_reported.xlsx', header=None, sheetname=country)
    dist = ages_dist[country].values
    M += C / dist
#     break
M /= 8
M = 0.5 * (M + M.T)
np.savetxt('./data/mossong/medlock_avg_sym.csv', M, fmt='%.5f', delimiter=',')

In [32]:
def medlock(C, vec):
    '''Takes a medlock symmetric Matrix with an ages vector, and stratifies the matrix according to the ages'''
    v = [np.arange(a,b) for a, b in zip(vec[:-1], vec[1:])]
    E = np.zeros([len(v)] * 2)
    for i in range(len(v)):
        for j in range(len(v)):
            val = 0
            cnt = 0
            for vi in v[i]:
                for vj in v[j]:
                    cnt+=1
                    val += C[min(C.shape[0] - 1, int(vi)//5), min(C.shape[1] - 1, int(vj)//5)]
            E[i, j] = val / cnt
    return E

In [34]:
# Relevant Ages
_ages = np.hstack((np.arange(0, 1, 2 / 12),  # First year: Every 2 months
                   np.arange(1, 14, 1),  # Every year until age of 13
                   15, 18, 21,  # High Scholl, Grad, Army
                   np.arange(25, 66, 10),  # 10 Year gaps
                   100))  # Death
_ages

array([   0.        ,    0.16666667,    0.33333333,    0.5       ,
          0.66666667,    0.83333333,    1.        ,    2.        ,
          3.        ,    4.        ,    5.        ,    6.        ,
          7.        ,    8.        ,    9.        ,   10.        ,
         11.        ,   12.        ,   13.        ,   15.        ,
         18.        ,   21.        ,   25.        ,   35.        ,
         45.        ,   55.        ,   65.        ,  100.        ])

In [53]:
med = medlock(M, _ages)
med

df = pd.DataFrame(med)
# df.to_csv('./data/test.csv', header=False, index=False)